In [ ]:
#https://etfdb.com/index/sp-500-index/
#https://finance.yahoo.com/research-hub/screener/SP_500_ETFS/
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/All_external.csv"
import pandas as pd
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from collections import Counter
from nltk.corpus import stopwords
import re
fnspid = pd.read_csv(path)

Mounted at /content/drive


<ipython-input-1-3b7952189c12>:16: DtypeWarning: Columns (2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  fnspid = pd.read_csv(path)


In [ ]:
# Load the FinBERT model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
fnspid.shape

(13057514, 11)

In [ ]:
sp500list = [
    "SPYU", "SPLG", "IVV", "SPYG", "SPYV", "VOO", "SPYD", "VOOG", "VOOV",
    "XVV", "SPXE", "SPXN", "SPXT", "SPXV", "SPY", "EFIV", "SNPE", "SPMV",
    "QVML", "SPMO", "SPVU", "SNPG", "SNPV", "SPHQ", "DSPY", "SPDN", "RSP",
    "SCHD", "IWD", "VYM", "JEPI", "IVE","AAPL", "MSFT", "NVDA", "AMZN",
    "META", "BRK.B", "GOOGL", "JPM", "TSLA","SPXU", "UPRO", "SH", "PSLD",
    "SPHB","BAC", "WFC", "C", "GS", "MS", "XOM", "CVX", "CAT", "BA", "JNJ",
    "UNH", "PFE", "MRK","WMT", "HD", "COST", "MCD", "DIS","ORCL",
    "ADBE", "CRM", "INTC", "AVGO","SUSA", "SPLV", "VTI"
]

fnspid_filtered = fnspid[fnspid['Stock_symbol'].isin(sp500list)]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
fnspid_filtered.head()

,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
6680,2020-06-10 07:33:26 UTC,Tech Stocks And FAANGS Strong Again To Start D...,AAPL,https://www.benzinga.com/government/20/06/1622...,JJ Kinahan,NaN,NaN,NaN,NaN,NaN,NaN
6681,2020-06-10 04:14:08 UTC,10 Biggest Price Target Changes For Wednesday,AAPL,https://www.benzinga.com/analyst-ratings/price...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN
6682,2020-06-10 03:53:47 UTC,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",AAPL,https://www.benzinga.com/short-sellers/20/06/1...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN
6683,2020-06-10 03:19:25 UTC,"Deutsche Bank Maintains Buy on Apple, Raises P...",AAPL,https://www.benzinga.com/news/20/06/16219873/d...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN
6684,2020-06-10 02:27:11 UTC,Apple To Let Users Trade In Their Mac Computer...,AAPL,https://www.benzinga.com/news/20/06/16218697/a...,Neer Varshney,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
fnspid_filtered.shape

(58266, 11)

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to get sentiment predictions using FinBERT
def get_sentiment_predictions(texts, batch_size=32):
    all_preds = []
    label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tokens = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        tokens = {k: v.to(device) for k, v in tokens.items()}

        with torch.no_grad():
            outputs = model(**tokens)
            preds = torch.argmax(outputs.logits, axis=1)
            preds = preds.cpu().numpy()
            all_preds.extend([label_map[p] for p in preds])

    return all_preds

# Apply FinBERT to the dataset and get predicted sentiments
fnspid_filtered['predicted_sentiment'] = get_sentiment_predictions(fnspid_filtered['Article_title'].tolist())


<ipython-input-8-334533e3f1f4>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fnspid_filtered['predicted_sentiment'] = get_sentiment_predictions(fnspid_filtered['Article_title'].tolist())


In [ ]:
save = '/content/drive/My Drive/sentiment.csv'

fnspid_filtered.to_csv(save, index=False)

In [ ]:
path = "/content/drive/My Drive/sentiment.csv"
df = pd.read_csv(path)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.head()


,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary,predicted_sentiment
0,2020-06-10 07:33:26+00:00,Tech Stocks And FAANGS Strong Again To Start D...,AAPL,https://www.benzinga.com/government/20/06/1622...,JJ Kinahan,NaN,NaN,NaN,NaN,NaN,NaN,negative
1,2020-06-10 04:14:08+00:00,10 Biggest Price Target Changes For Wednesday,AAPL,https://www.benzinga.com/analyst-ratings/price...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN,positive
2,2020-06-10 03:53:47+00:00,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",AAPL,https://www.benzinga.com/short-sellers/20/06/1...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN,positive
3,2020-06-10 03:19:25+00:00,"Deutsche Bank Maintains Buy on Apple, Raises P...",AAPL,https://www.benzinga.com/news/20/06/16219873/d...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN,negative
4,2020-06-10 02:27:11+00:00,Apple To Let Users Trade In Their Mac Computer...,AAPL,https://www.benzinga.com/news/20/06/16218697/a...,Neer Varshney,NaN,NaN,NaN,NaN,NaN,NaN,positive


In [ ]:
dummies = pd.get_dummies(df['predicted_sentiment'], prefix='sentiment')

df = pd.concat([df, dummies], axis=1)


In [ ]:
df.head()

,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary,predicted_sentiment,sentiment_negative,sentiment_neutral,sentiment_positive
0,2020-06-10 07:33:26+00:00,Tech Stocks And FAANGS Strong Again To Start D...,AAPL,https://www.benzinga.com/government/20/06/1622...,JJ Kinahan,NaN,NaN,NaN,NaN,NaN,NaN,negative,True,False,False
1,2020-06-10 04:14:08+00:00,10 Biggest Price Target Changes For Wednesday,AAPL,https://www.benzinga.com/analyst-ratings/price...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN,positive,False,False,True
2,2020-06-10 03:53:47+00:00,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",AAPL,https://www.benzinga.com/short-sellers/20/06/1...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN,positive,False,False,True
3,2020-06-10 03:19:25+00:00,"Deutsche Bank Maintains Buy on Apple, Raises P...",AAPL,https://www.benzinga.com/news/20/06/16219873/d...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN,negative,True,False,False
4,2020-06-10 02:27:11+00:00,Apple To Let Users Trade In Their Mac Computer...,AAPL,https://www.benzinga.com/news/20/06/16218697/a...,Neer Varshney,NaN,NaN,NaN,NaN,NaN,NaN,positive,False,False,True


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(df['predicted_sentiment'])

df['sentiment_encoded'] = labels

df.head()


,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary,predicted_sentiment,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment_encoded
0,2020-06-10 07:33:26+00:00,Tech Stocks And FAANGS Strong Again To Start D...,AAPL,https://www.benzinga.com/government/20/06/1622...,JJ Kinahan,NaN,NaN,NaN,NaN,NaN,NaN,negative,True,False,False,0
1,2020-06-10 04:14:08+00:00,10 Biggest Price Target Changes For Wednesday,AAPL,https://www.benzinga.com/analyst-ratings/price...,Lisa Levin,NaN,NaN,NaN,NaN,NaN,NaN,positive,False,False,True,2
2,2020-06-10 03:53:47+00:00,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",AAPL,https://www.benzinga.com/short-sellers/20/06/1...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN,positive,False,False,True,2
3,2020-06-10 03:19:25+00:00,"Deutsche Bank Maintains Buy on Apple, Raises P...",AAPL,https://www.benzinga.com/news/20/06/16219873/d...,Benzinga Newsdesk,NaN,NaN,NaN,NaN,NaN,NaN,negative,True,False,False,0
4,2020-06-10 02:27:11+00:00,Apple To Let Users Trade In Their Mac Computer...,AAPL,https://www.benzinga.com/news/20/06/16218697/a...,Neer Varshney,NaN,NaN,NaN,NaN,NaN,NaN,positive,False,False,True,2


In [ ]:
output_path = '/content/drive/My Drive/sentiment_data.csv'

df.to_csv(output_path, index=False)
